Что нужно сделать далее:
1. Рисовать графики/ Можно и точки, и линии (точки удобнее к настройке), получение данных есть
2. Изменение параметров пласта
3. КПД
4. КВД
5. Непроницаемая граница
6. Постоянное давление на контуре
7. Диагностический график
8. Добавление классов для каждого графика свои данные!!!
9. Скин фактор
10. ВСС
11. Алгорит Стехфеста
12. Изменение давления в безразмерных координатах
13. Сравнение КВД и КПД
14. Гидропрослушивание
15. Логарифмическая аппроксимация имеет место при...
16. Эталонная кривая

.
.
.
99. Интепретатор ГДИС
.
.
.
999. CRM
1000. Термометрия

ИСП. Графики

Импортируем все необходимые модули

In [74]:

#для bokeh
from bokeh.layouts import column,row,layout
from bokeh.models import ColumnDataSource, Slider, RadioButtonGroup,TableColumn, DataTable
from bokeh.plotting import figure, output_file, show
from bokeh.themes import Theme
from bokeh.io import show, output_notebook, output_file, push_notebook
from bokeh.layouts import widgetbox
from bokeh.models.tools import HoverTool, PointDrawTool
#прочие
import pandas as pd
import numpy as np
from scipy import special
import os
import scipy as sp
import matplotlib.pyplot as plt
import pylab




output_notebook() #для вывода в Jupyter Notebook

Loading BokehJS ...

Зададим исходные данные

In [18]:
k_md=10
mu_sp=1
ct_1atm=5*10**(-5)
rw_m=0.1
q_m3day=10
B_m3m3=1.2
Pi_bar=250
m=0.2
h_m=1

Создадим основные и вспомогательные функции

In [99]:

def out(string1, result,string2=''): #выпод вычислений с комментариями
    space='  '
    print(string1 + space + str(result) + space + string2 + '\n')


#Инегральная экспонента и логарифмическая аппроксимация

Ei=sp.special.expi  #f=-Ei(-x), используется в уравнении решения фильтрации
E1=sp.special.exp1 #f=E1(x);   E1(x)=-Ei(-x)=-ln(x)-gammd
ln=np.log

def LogApprox(x): #логарифмическая аппроксимация, только для числа, не списка
    gamma=0.57
    #gamma=0.809
    #result=(-1)*math.log(x)-gamma
    result=(-1)*ln(x)-gamma
    return result


def genvaluesx(k,x0=0.001): #генерация значений x c множителем к
    values=[float(x0)] #начальное значение
    for i in range(20): #число элементов 
        values.append(values[i]*k)
    result=np.asanyarray(values)
    return result

def save(name='', fmt='png'): #в текущей директории должна быть папка pictures
    pwd = os.getcwd()
    iPath = './{}'.format(fmt)
    if not os.path.exists(iPath):
        os.mkdir(iPath)
    os.chdir(iPath)
    plt.savefig('{}.{}'.format(name, fmt), fmt='png')
    os.chdir(pwd)
    

def td(t,k=k_md,m=m,mu=mu_sp,ct=ct_1atm,rw=rw_m): #безразмерное время
    result=0.00036*k*t/(m*mu*ct*rw*rw)
    return result

def pd_E1(rd,td): #безразмерное давление через интегрально экспоненциальную функцию
    result=1/2*E1(rd**2/4/td)
    return result

#??? какая из функций
def pd_ln(rd,td): #безравзерное давление через логарифм
    #result=1/2*LogApprox(rd**2/4/td)
    result=1/2*(ln(td/rd**2)+0.80907)
    return result

def rd(r,rw=rw_m): #безразмерное расстояние
    result=r/rw
    return result

def pwf(pd,k=k_md,h=h_m,q=q_m3day,b=B_m3m3,mu=mu_sp,pi=Pi_bar): #преобразование безразмерного давления в размерное
    result=pi-18.41*pd*q*b*mu/k/h
    return result

   
def p(r,t,mod=1): #решение линейного стока 
    td1=td(t)
    rd1=rd(r)
    if mod==1: #решение линейного стока с помощью интегральной экспоненты
        pd1=pd_E1(rd1,td1)
        result=pwf(pd1)
        return result
    if mod==2: #решение линейного стока с помощью логарфима
        pd1=pd_ln(rd1,td1)
        result=pwf(pd1)
        return result       

def mistake(a,b): #вычисление относительной ошибки
    result=(a-b)/a*100    
    return np.absolute(result)
    


 


Выбор типа осей для построения ВСЕХ графиков. Для активации необходимо перезапустить ячейку с графиком

In [20]:
#параметры, устанавливающие тип осей для ВСЕХ графиков, меняются через функцию(кнопки)
ytype='linear' 
xtype='linear'
def choiceaxis(): #вывод кнопок для смены типов осей
    def modify_doc(doc):
        def callback2(attr, old, new):
            global xtype
            global ytype 
            if new==0:
                ytype='linear'
                xtype='linear'
            if new==1:
                ytype='linear'
                xtype='log'
            if new==2:
                ytype='log'
                xtype='linear'       
            if new==3:
                ytype='log'
                xtype='log'

        button_group = RadioButtonGroup(labels=["y=f(x)", "y=ln(f(x))", "ln(y)=f(x)",'ln(y)=ln(f(x))'], active=0)
        button_group.on_change('active', callback2)
        doc.add_root(button_group)       
    show(modify_doc)
choiceaxis()

Интегральная экспонента  и ее логарифмическая аппроксимация

In [21]:
#исходные данные для построения графика
valuesx=genvaluesx(1.1)
data33 = {'x_values': valuesx,
        'y_values': E1(valuesx),
        'y_values2': LogApprox(valuesx),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx))}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)    #место, где хранятся данные фигуры
    plot1 = figure(title="E и Ln", x_axis_label='x', y_axis_label='E1(x) и Ln(x)',x_axis_type=xtype, y_axis_type=ytype)
    def plotline():
        plot1.line('x_values', 'y_values', color="red", source=source2, legend='E(x)',name='E(x)')
        plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='-ln(x)-0.57',name='Ln(x)')
        #plot1.line('x_values', 'mistake', color="green", source=source2, legend='Ошибка',name='Ошибка')
    plotline()
    def callback(attr, old, new): #при выполнении события для слайдера
        data=data2.copy()
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new)
            data['x_values']=valuesx
            data['y_values']=E1(valuesx)
            data['y_values2']=LogApprox(valuesx)       
        source2.data = ColumnDataSource(data=data).data  
    slider = Slider(start=1.1, end=2, value=1.1, step=0.1, title="Множитель к, расширяющий ось x") #отображение слайдера
    slider.on_change('value', callback) #события для слайдера, вызывающее функции
    doc.add_root(layout([[slider],[plot1]]))
    
show(modify_doc)

Решение линейного стока p(t)

In [22]:
choiceaxis()

In [23]:
t0=2*10**(-7)
valuesx=genvaluesx(1.3,t0)

data33 = {'x_values': valuesx,
        'y_values':p(0.1,valuesx),
        'y_values2': p(0.1,valuesx,2),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx))}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(t), r=0.1", x_axis_label='t', y_axis_label='P',x_axis_type=xtype, y_axis_type=ytype)
    def plotline():
        plot1.line('x_values', 'y_values', color="red", source=source2, legend='через E(x)',name='E(x)')
        plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
    plotline()
    def callback(attr, old, new):
        data=data2.copy()
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,t0)
            data['x_values']=valuesx
            data['y_values']=p(0.1,valuesx)
            data['y_values2']=p(0.1,valuesx,2)
        source2.data = ColumnDataSource(data=data).data  
    slider = Slider(start=1.1, end=2, value=1.3, step=0.1, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    doc.add_root(layout([[slider],[plot1]]))

    
show(modify_doc)

In [24]:
choiceaxis()

Решение линейного стока #3 p(r,t)

In [25]:
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
#ymistake=mistake(E1(valuesx),LogApprox(valuesx))
data33 = {'x_values': valuesx,
        'y_values':p(valuesx,t),
        'y_values2': p(valuesx,t,2),
         'mistake': mistake(E1(valuesx),LogApprox(valuesx)),
         'initialp': valuesx*0+Pi_bar}

data2 = pd.DataFrame(data33)

def modify_doc(doc):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
    
    plot1.line('x_values', 'y_values', color="red", source=source2, legend='через E(x)',name='E(x)')
    plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
    plot1.line('x_values', 'initialp', color="green", source=source2, legend='Pi',name='Pi')
    
    def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  
    
    slider = Slider(start=1.1, end=2, value=1.3, step=0.05, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    slidert = Slider(start=1/(10**3), end=100, value=0.01, step=0.01, title="Время")
    slidert.on_change('value', callback2)
    doc.add_root(layout([[slider,slidert],[plot1]]))

    
show(modify_doc)

In [68]:
print(source22.data)

NameError: name 'source22' is not defined

In [51]:
from bokeh.plotting import figure, output_file, show, Column
from bokeh.models import DataTable, TableColumn, PointDrawTool, ColumnDataSource

#output_file("tools_point_draw.html")

p = figure(x_range=(0, 10), y_range=(0, 10), tools=[],
           title='Point Draw Tool')
p.background_fill_color = 'lightgrey'

source = ColumnDataSource({
    'x': [1, 5, 9], 'y': [1, 5, 9], 'color': ['red', 'green', 'yellow']
})

renderer = p.scatter(x='x', y='y', source=source, color='color', size=10)
columns = [TableColumn(field="x", title="x"),
           TableColumn(field="y", title="y"),
           TableColumn(field='color', title='color')]
table = DataTable(source=source, columns=columns, editable=True, height=200)

draw_tool = PointDrawTool(renderers=[renderer], empty_value='black')
p.add_tools(draw_tool)
p.toolbar.active_tap = draw_tool

show(Column(p, table))

In [78]:
# myapp.py

from random import random

from bokeh.layouts import column
from bokeh.models import Button
from bokeh.palettes import RdYlBu3
from bokeh.plotting import figure, curdoc

# create a plot and style its properties
p = figure(x_range=(0, 100), y_range=(0, 100), toolbar_location=None)
p.border_fill_color = 'black'
p.background_fill_color = 'black'
p.outline_line_color = None
p.grid.grid_line_color = None

# add a text renderer to our plot (no data yet)
r = p.text(x=[], y=[], text=[], text_color=[], text_font_size="20pt",
           text_baseline="middle", text_align="center")

i = 0

ds = r.data_source

# create a callback that will add a number in a random location
def callback():
    global i

    # BEST PRACTICE --- update .data in one step with a new dict
    new_data = dict()
    new_data['x'] = ds.data['x'] + [random()*70 + 15]
    new_data['y'] = ds.data['y'] + [random()*70 + 15]
    new_data['text_color'] = ds.data['text_color'] + [RdYlBu3[i%3]]
    new_data['text'] = ds.data['text'] + [str(i)]
    ds.data = new_data

    i = i + 1

# add a button widget and configure with the call back
button = Button(label="Press Me")
button.on_click(callback)

# put the button and plot in a layout and add to the document
curdoc().add_root(column(button, p))
output_notebook()
show(Column(p, button), notebook_handle=True)
push_notebook()

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [107]:
class Vehicle:
    """docstring"""
    
    def __init__(self,b):
        """Constructor"""
        #self.attr='lol'
        
    def lie(self):
        self.attr='lol'
        print('Метод, реализованный без параметров')

a=Vehicle
a.lie(a)
b=a.attr
print(b)

class Dog:
    name = "Charlie"
    noise = "Woof!"
    def makeNoise(self):
        print(self.name + " says: " + self.noise + " " + self.noise)
dog = Dog()
dog.makeNoise()

Метод, реализованный без параметров
lol
Charlie says: Woof! Woof!


In [161]:
datasample={'x':[1,2,3],
        'y':[1,2,3]}
datasample = pd.DataFrame(datasample)
class Plotting:
    output_notebook()
    datadraw={'x':[],
              'y':[]}
    ytype='linear'
    xtype='linear'
    x_axis='x'
    y_axis='y'
    title='Title'
    source=None
    sourcedraw=ColumnDataSource(datadraw)
    plot=figure(title=title, x_axis_label=x_axis, y_axis_label=y_axis,x_axis_type=xtype, y_axis_type=ytype)
    plot.scatter('x','y',source=source)
    
    def __init__(self,data):
        self.source=ColumnDataSource(data=data)
    
    #def standatd():
    #    self.bfigure=figure
        
    def modify_doc(doc):
        doc.add_root(layout[[plot]])
        #show(modify_doc)
    
    def start():
        #plot.scatter('x','y',source=source)
        
        #show(modify_doc)
        show(modify_doc)
#doc.add_root()    
A=Plotting(datasample)    
print(A.source.data)
A.modify_doc()

Loading BokehJS ...

{'index': array([0, 1, 2], dtype=int64), 'x': array([1, 2, 3], dtype=int64), 'y': array([1, 2, 3], dtype=int64)}


AttributeError: 'Plotting' object has no attribute 'add_root'

In [126]:
def aa(bb):
    bb=bb
    print(bb)
    def cc(bb):
        print(bb)
print(aa(2))

2
None


Отличия от класса:
1.  x, y 
2. смена парамеров осей
3. добавление инструментов
4. 

In [128]:
#2 source, один для основных данных, другой для точек
#ограничение на 2 точки
#нужен класс для сохранения данных
#прочие приколюхи tools на графике
#класс для всего

#  отличия для класса:
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
#ymistake=mistake(E1(valuesx),LogApprox(valuesx))
data334 = {'x_values': valuesx,
          'y_values2': p(valuesx,t),
          'y_values22': p(valuesx,t,2),
          'mistake2': mistake(E1(valuesx),LogApprox(valuesx)),
          'initialp2': valuesx*0+Pi_bar,
          'x':valuesx*0,
          'y':valuesx*0}
data333={'x':[],
        'y':[]}
data22 = pd.DataFrame(data333)
data2 = pd.DataFrame(data334)

def modify_doc(doc):
    source22 = ColumnDataSource(data=data22) 
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
   # plot1.title(" rw=0.1")
    
    plot1.line('x_values', 'y_values2', color="red", source=source2, legend='через E(x)',name='E(x)')
#    plot1.line('x_values', 'y_values2', color="blue", source=source2, legend='через ln(x)',name='Ln(x)')
#    plot1.line('x_values', 'initialp', color="green", source=source2, legend='Pi',name='Pi')
    r1 = plot1.scatter('x', 'y',marker='circle',radius=5, source=source22)

    plot1.add_tools(PointDrawTool(renderers=[r1], empty_value = 1,num_objects=2))
    columns=[TableColumn(field='x',title='x'),
            TableColumn(field='y',title='y')]
    table=DataTable(source=source22,columns=columns, editable=True)
    print(TableColumn(field='x',title='x'))
    def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        print(source22.data['x'][0]-source22.data['x'][1])
        a=source22.data['x'][0]-source22.data['x'][1]
        print(a)
        print(type(a))
        plot1.line('x', 'y', color="red", source=source22, legend='через   E(x)',name='E(x)')
        
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  
    
    slider = Slider(start=1.1, end=2, value=1.3, step=0.05, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    slidert = Slider(start=1/(10**3), end=100, value=0.01, step=0.01, title="Время")
    slidert.on_change('value', callback2)
    doc.add_root(layout([[slider,slidert],[plot1],[table]]))
    
    
show(modify_doc)

TableColumn(id='28338', ...)
-12.08831045885682
-12.08831045885682
<class 'float'>


In [29]:
a=[0,0,0]
b=[0,0,0]
c=[0,1,2]
d=[0,1,2]
r0=0.1
t=1
valuesx=genvaluesx(1.3,r0)
data33 = {'x':a,
         'y':b,
         'y2':c,
         'x2':d}

data2 = pd.DataFrame(data33)

source = ColumnDataSource(data2) 
p=figure()
p.rect('x2','y2',0.1, 0.1,source=source)
#c1 = p.circle('x', 'y', source=source)
r1 = p.rect('x', 'y', 0.1, 0.1, source=source)
#tool = PointDrawTool(renderers=[c1, r1])
#plot = figure(tools="pan,wheel_zoom,box_zoom,reset")
p.add_tools(PointDrawTool(renderers=[r1]))
show(p)

def modify_doc2(doc22):
    source2 = ColumnDataSource(data=data2)  
    plot1 = figure(title="P(r,t), rw=0.1", x_axis_label='r,м', y_axis_label='P, атм',x_axis_type=xtype, y_axis_type=ytype)
    
    plot1.line('x', 'y', color="red", source=source2, legend='через E(x)',name='E(x)')
    r1 = plot1.rect('x', 'y', 0.1, 0.1, source=source)

    p.add_tools(PointDrawTool(renderers=[r1]))
   
    '''def callback(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            valuesx=genvaluesx(new,r0)
            data['initialp']=valuesx*0+Pi_bar
            data['x_values']=valuesx
            data['y_values']=p(valuesx,t)
            data['y_values2']=p(valuesx,t,2)
        source2.data = ColumnDataSource(data=data).data  
    
    def callback2(attr, old, new):
        data=data2
        if new == 0:
            data=data2.copy()
        else:
            data['y_values']=p(data['x_values'],new)
            data['y_values2']=p(data['x_values'],new,2)
        source2.data = ColumnDataSource(data=data).data  '''
    
    doc22.add_root(layout([[plot1]]))
    

    
show(modify_doc2)

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=11279&bokeh-absolute-url=http://localhost:52574&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:52574', method='GET', uri='/autoload.js?bokeh-autoload-element=11279&bokeh-absolute-url=http://localhost:52574&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\web.py", line 1592, in _execute
    result = yield result
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\gen.py", line 1133, in run
    value = future.result()
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\tornado\gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\ProgramData\PPythonAnacondaOneLove\lib\site-packages\bokeh\server\views\autoload_js_handler.py", line 60, in get
    session = yield self.get_session()
  File "C:\ProgramData\PPythonAnacon

In [12]:
print(source.data)

{'index': array([0, 1, 2], dtype=int64), 'x': array([0, 0, 0], dtype=int64), 'y': array([0, 0, 0], dtype=int64), 'y2': array([0, 1, 2], dtype=int64), 'x2': array([0, 1, 2], dtype=int64)}


Полигон для проверки функций #3

In [27]:
data33 = {'x_values': valuesx,
        'y_values': solvE1}
data2 = pd.DataFrame(data33)

def modify_doc(doc):
   
    source2 = ColumnDataSource(data=data2)
    output_notebook()
    plot1 = figure(title="P(t), r=0.1 м", x_axis_label='t, часы', y_axis_label='P по Е1')
    plot1.line('x_values', 'y_values', source=source2)

    def callback(attr, old, new):
        data=data2.copy()
        if new == 0:
            data=data
        else:
            valuesx=genvaluesx(new)
            valuesx=np.asanyarray(valuesx)
            solvE1=p(0.1,valuesx)
            data333 = {'x_values': valuesx,'y_values': solvE1}
            data22 = pd.DataFrame(data333)
            
            data['x_values']=data22['x_values']
            data['y_values']=data22['y_values']
        source2.data = ColumnDataSource(data=data).data
    
    slider = Slider(start=1, end=2, value=1.1, step=0.1, title="Множитель к, расширяющий ось x")
    slider.on_change('value', callback)
    
    doc.add_root(column(slider, plot1))
    


    
show(modify_doc)

NameError: name 'solvE1' is not defined

In [ ]:

x=0.001
k=1.4 #множитель по оси x


td1=td(83.336111111)
rd1=rd(0.1)

valuesx=genvaluesx(k)
valuesx=np.asanyarray(valuesx)
print(p(0.1,0.00001))
print(p(0.1,0.00001,2))
solvE1=p(0.1,valuesx)
solvLog=p(0.1,valuesx,2)

out('-Ei(-x)=',-Ei(-x), 'Пример значения интегральной экспоненты при x={}'.format(str(x)))
out('E1(x)=',E1(x), 'E1(x)=-Ei(-x)=-ln(x)-gamma при x={}'.format(str(x)))
out('-ln(x)-gamma=',LogApprox(x),'логарифмическая аппроксимация интегральной экспоненты при x={}'.format(str(x)))


#figsize = (4,3)
#fig=plt.figure(figsize=figsize, dpi=150, facecolor='green', frameon=True)
fig=plt.figure()
#plt.xscale('log') 
plt.plot(valuesx,solvE1, label='E1')
plt.plot(valuesx,solvLog, label='Ln')

plt.title('Интегральная экспонента и ее аппроксимация')   # заголовок
plt.xlabel('x')   # подпись оси OX
plt.ylabel('E1(x); -ln(x)-0.57')   # подпись оси OY
plt.legend()
plt.grid(True)

save('pic_1')
plt.show()